# Reading Dataset

In [1]:
import sqlite3
import random

conn = sqlite3.connect('Data/data.db')
c = conn.cursor()

query = c.execute("SELECT author || '-' || title, text FROM lyrics")

lyrics = []

for row in query:
    lyrics.append(row)

conn.close()

random.shuffle(lyrics)

lyrics = lyrics[0:10000]

# Train / Test Split

In [2]:
from sklearn.model_selection import train_test_split

lyrics_train, lyrics_test = train_test_split(lyrics, test_size=0.30)

del lyrics

print("len(lyrics_train) = %d" % len(lyrics_train))
print("len(lyrics_test) = %d" % len(lyrics_test))

len(lyrics_train) = 7000
len(lyrics_test) = 3000


# Setting Parameters

In [3]:
N_TOKENS = 4
N_HASHES = 100
N_BANDS = 10
N_BUCKETS = 2**64
KEY = 1
TRUTH_KEY = 0

# Phase 1 - Shingles

In [4]:
from nltk import ngrams
import binascii
import numpy as np

def to_shingles(lyrics, n_tokens, key, silent=False):
    shingles = []

    for i, lyric in enumerate(lyrics):
        features = list(ngrams(list(lyric[key]), n_tokens))
        # Hashing
        shingle = [binascii.crc32(" ".join(feature)) & 0xffffffff for feature in features]
        
        if len(shingle) == 0:
            print("WARNING: skipping empty shingle (i = %d)" % i)
            continue
        
        shingle.sort()
        shingle = np.array(shingle, dtype=np.int)
        shingles.append(shingle)

        if not silent and i > 0 and i % (len(lyrics) / 10) == 0:
            print("%d of %d" % (i, len(lyrics)))

    if not silent:
        print("Done!")
    
    return shingles

In [5]:
%time shingles = to_shingles(lyrics_test, n_tokens=N_TOKENS, key=KEY)

300 of 3000
600 of 3000
900 of 3000
1200 of 3000
1500 of 3000
1800 of 3000
2100 of 3000
2400 of 3000
2700 of 3000
Done!
CPU times: user 4.44 s, sys: 230 ms, total: 4.67 s
Wall time: 4.33 s


# Phase 2 - Min Hashing

In [6]:
def to_signatures(shingles, n_hashes, silent=False):
    M = 4294967311

    a = np.random.randint(2 ** 32, size=(n_hashes, 1)) # n_hashes x 1
    b = np.random.randint(2 ** 32, size=(n_hashes, 1)) # 1 x n_hashes
    
    signatures = np.zeros((len(shingles), n_hashes), dtype=np.int) # len(shingles) x n_hashes

    for i, shingle in enumerate(shingles):
        tmp = (a * shingle + b) % M # n_hashes x len(shingle)
        signature = np.min(tmp, axis=1)
        signatures[i] = signature

        if not silent and i > 0 and i % (len(shingles) / 10) == 0:
            print("%d of %d" % (i, len(shingles)))

    if not silent:
        print("Done!")
    
    return signatures

In [7]:
%time signatures = to_signatures(shingles, n_hashes=N_HASHES)

del shingles

300 of 3000
600 of 3000
900 of 3000
1200 of 3000
1500 of 3000
1800 of 3000
2100 of 3000
2400 of 3000
2700 of 3000
Done!
CPU times: user 5.91 s, sys: 7.69 ms, total: 5.92 s
Wall time: 5.94 s


# Phase 3 - LSH

In [8]:
import random

def generate_h(M):
    a = random.randint(0, M - 1)
    b = random.randint(0, M - 1)

    return lambda x: (a * x + b) % M

def signature_hash(signature, start, end, n_buckets):
    M = 4294967311
    k = 0

    for e in signature[start:end]:
        k = (k + int(e) * M) % n_buckets
    
    return k

def lsh(signatures, n_bands, n_buckets, silent=False):
    rows = len(signatures[0]) / n_bands # rows per bucket

    buckets = []
    
    for i in xrange(n_bands):
        h = generate_h(n_buckets) # for each band, create a hash function
        
        band_buckets = {}
        
        start = rows * i

        if i == n_bands - 1:
            end = len(signature)
        else:
            end = rows * (i + 1)
        
        x = np.random.randn(1, len(signatures[0][start:end]))
        b = np.random.random()
    
        for j, signature in enumerate(signatures):
            k = signature_hash(signature, start, end, n_buckets)
            
            if k not in band_buckets:
                band_buckets[k] = []
            
            band_buckets[k].append(j)
        
        for band_bucket in band_buckets.values():
            if len(band_bucket) <= 1:
                continue
            
            buckets.append(np.array(band_bucket, dtype=np.int))
        
        if not silent:
            print("%d of %d" % (i, n_bands))
    
    if not silent:
        print("Done!")
        print("Average bucket length = %.2lf" % np.mean([len(bucket) for bucket in buckets if len(bucket) > 0]))
        print("Median bucket length = %.2lf" % np.median([len(bucket) for bucket in buckets if len(bucket) > 0]))
        print("Max bucket length = %.2lf" % np.max([len(bucket) for bucket in buckets]))
    
    return buckets

In [9]:
%time buckets = lsh(signatures, n_bands=N_BANDS, n_buckets=N_BUCKETS)

0 of 10
1 of 10
2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
Done!
Average bucket length = 2.28
Median bucket length = 2.00
Max bucket length = 10.00
CPU times: user 209 ms, sys: 21.1 ms, total: 230 ms
Wall time: 193 ms


# Detecting Duplicates with LSH

In [10]:
def get_false_negatives(lyrics, key):
    false_negatives = {}

    for row in lyrics:
        false_negatives[row[key]] = false_negatives.get(row[key], 0) + 1
    
    false_negatives_keys = false_negatives.keys()

    for k in false_negatives_keys:
        false_negatives[k] = (false_negatives[k] * (false_negatives[k] - 1)) / 2
        
        if false_negatives[k] == 0:
            del false_negatives[k]

    return sum(false_negatives.values())

def dup_lsh(buckets, signatures, lyrics, key, truth_key, silent=False):
    tp, fp = 0, 0
    fn = get_false_negatives(lyrics, TRUTH_KEY)
    
    tested = {}
    
    for i, bucket in enumerate(buckets):
        for j in bucket:
            for k in bucket:
                if k <= j or (j, k) in tested or (k, j) in tested:
                    continue
                
                same_key = lyrics[j][truth_key] == lyrics[k][truth_key]
                
                if same_key:
                    tp += 1
                    fn -= 1
                else:
                    fp += 1
                
                tested[j, k] = True
        
        if not silent and i > 0 and i % (len(buckets) / 10) == 0:
            print("%d of %d" % (i, len(buckets)))

    duplicates = tp + fp
    precision = tp / float(tp + fp)
    recall = tp / float(tp + fn)
    
    if not silent:
        print("Done!")
        print("dup=%d, tp=%d, fp=%d, fn=%d, prec=%.2lf, rec=%.2lf" %
              (duplicates, tp, fp, fn, precision, recall))
    
    return duplicates, tp, fp, fn, precision, recall

In [11]:
%time _ = dup_lsh(buckets, signatures, lyrics_test, key=KEY, truth_key=TRUTH_KEY)

43 of 431
86 of 431
129 of 431
172 of 431
215 of 431
258 of 431
301 of 431
344 of 431
387 of 431
430 of 431
Done!
dup=117, tp=36, fp=81, fn=3, prec=0.31, rec=0.92
CPU times: user 8.48 ms, sys: 141 µs, total: 8.62 ms
Wall time: 6.61 ms


# Parameter Calibration

In [ ]:
def grid_search(lyrics, key, truth_key):
    N_BUCKETS = 2 ** 64

    for N_TOKENS in [5, 4, 6, 7, 8]:
        for N_HASHES in [100, 200, 300]:
            for N_BANDS in [12, 14, 16, 18, 20]:
                print("N_TOKENS=%d, N_HASHES=%d, N_BANDS=%d, N_BUCKETS=%d" % (N_TOKENS, N_HASHES,
                                                                              N_BANDS, N_BUCKETS))
                shingles = to_shingles(lyrics, n_tokens=N_TOKENS, key=KEY, silent=True)
                signatures = to_signatures(shingles, n_hashes=N_HASHES, silent=True)
                del shingles
                buckets = lsh(signatures, n_bands=N_BANDS, n_buckets=N_BUCKETS, silent=True)
                duplicates, tp, fp, fn, precision, recall = dup_lsh(buckets, signatures, lyrics,
                                                                    key=KEY, truth_key=TRUTH_KEY, silent=True)
                print("dup=%d, tp=%d, fp=%d, fn=%d, prec=%.2f, rec=%.2f" %
                      (duplicates, tp, fp, fn, precision, recall))

%time grid_search(lyrics_train, key=KEY, truth_key=TRUTH_KEY)